# Convolutional Neural Network Example

We build a simple, 1 convolutional and 1 dense layer neural  net and test it on MNIST dataset.


## Preliminaries

### Imports

In [1]:
import pickle

import numpy as np
import tensorflow as tf
from tensorflow import keras

# Helper libraries
import numpy as np
import matplotlib.pyplot as plt

print(tf.__version__)

import matplotlib.pyplot as plt

%matplotlib inline

import sys
sys.path.append("..")
import E4525_ML.mnist as mnist
from E4525_ML.TFClassifier import TFClassifier

/home/manel/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


1.11.0


In [2]:
#
from tensorflow.keras.backend import set_session

config = tf.ConfigProto()
config.gpu_options.allow_growth = True  # dynamically grow the memory used on the GPU
config.log_device_placement = True  # to log device placement (on which device the operation ran)
                                    # (nothing gets printed in Jupyter, only if you run it standalone)

sess = tf.Session(config=config)

set_session(sess)  # set this TensorFlow session as the default session for Keras

In [3]:
raw_data_dir="../raw/MNIST"
data_dir="../data/digits"

In [4]:
images_filename=raw_data_dir+"/train-images-idx3-ubyte.gz"
labels_filename=raw_data_dir+"/train-labels-idx1-ubyte.gz"

test_images_filename=raw_data_dir+"/t10k-images-idx3-ubyte.gz"
test_labels_filename=raw_data_dir+"/t10k-labels-idx1-ubyte.gz"

images=mnist.read_images(images_filename)
labels=mnist.read_labels(labels_filename)

test_images=mnist.read_images(test_images_filename)
test_labels=mnist.read_labels(test_labels_filename)
    
print(images.shape,labels.shape,test_images.shape,test_labels.shape)

(60000, 28, 28) (60000,) (10000, 28, 28) (10000,)


In [5]:
model = keras.Sequential([
    # keras needs  (Rows x Cols x Channels)
    # we must pass input shape, summaries will not work later
    keras.layers.Reshape((28,28,1),input_shape=(28,28)),
    
    
    # Convolutional Layers
    keras.layers.Conv2D(10, (5, 5), activation='relu',padding="same"),
    keras.layers.MaxPooling2D(pool_size=(2, 2)),
    keras.layers.Conv2D(20, (5, 5), activation='relu',padding="same"),
    keras.layers.MaxPooling2D(pool_size=(2, 2)),
    keras.layers.Dropout(0.5),
    
    # Dense Layers
    keras.layers.Flatten(),
    keras.layers.Dense(50, activation=tf.nn.relu),
    keras.layers.Dense(10, activation=tf.nn.softmax)
])

In [6]:
model.compile(optimizer=tf.train.AdamOptimizer(), 
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [7]:

print('Trainable parameters: {:,}'.format(model.count_params()))

Trainable parameters: 54,840


In [8]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
reshape (Reshape)            (None, 28, 28, 1)         0         
_________________________________________________________________
conv2d (Conv2D)              (None, 28, 28, 10)        260       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 14, 14, 10)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 14, 14, 20)        5020      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 7, 7, 20)          0         
_________________________________________________________________
dropout (Dropout)            (None, 7, 7, 20)          0         
_________________________________________________________________
flatten (Flatten)            (None, 980)               0         
__________

In [9]:
model.fit(images, labels, epochs=10,validation_split=0.2)

Train on 48000 samples, validate on 12000 samples
Epoch 1/10
48000/48000 [==============================] - 51s 1ms/step - loss: 0.2447 - acc: 0.9237 - val_loss: 0.0694 - val_acc: 0.9805
Epoch 2/10
48000/48000 [==============================] - 49s 1ms/step - loss: 0.0944 - acc: 0.9703 - val_loss: 0.0500 - val_acc: 0.9847
Epoch 3/10
48000/48000 [==============================] - 46s 951us/step - loss: 0.0750 - acc: 0.9765 - val_loss: 0.0472 - val_acc: 0.9853
Epoch 4/10
48000/48000 [==============================] - 46s 958us/step - loss: 0.0635 - acc: 0.9795 - val_loss: 0.0402 - val_acc: 0.9887
Epoch 5/10
48000/48000 [==============================] - 47s 980us/step - loss: 0.0571 - acc: 0.9812 - val_loss: 0.0368 - val_acc: 0.9897
Epoch 6/10
48000/48000 [==============================] - 46s 962us/step - loss: 0.0534 - acc: 0.9832 - val_loss: 0.0349 - val_acc: 0.9896
Epoch 7/10
48000/48000 [==============================] - 47s 980us/step - loss: 0.0486 - acc: 0.9842 - val_loss: 0.0325

In [10]:
Y_pred=np.argmax(model.predict(test_images),axis=1)

In [11]:
np.mean(Y_pred==test_labels)

0.9923

## Classification of my own hand-written digits.

In [16]:
digits_filename=data_dir+"/digits.p"
digits_file=open(digits_filename,"rb")
digits,digit_labels=pickle.load(digits_file)

In [17]:
# Use the model to predict the images class
Y_pred  = np.argmax(model.predict(digits),axis=1)
print(np.mean(Y_pred == digit_labels))

0.88


In [18]:
antialiased_digits_filename=data_dir+"/antialiased_digits.p"
antialiased_digits_file=open(antialiased_digits_filename,"rb")
antialiased_digits,antialiased_labels=pickle.load(antialiased_digits_file)

In [19]:
# Use the model to predict the images class
Y_pred  = np.argmax(model.predict(antialiased_digits),axis=1)
print(np.mean(Y_pred == antialiased_labels))

0.89
